<a href="https://colab.research.google.com/github/dahwi/deepLearning/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt

# Define transformations
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
trainset = datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
# Split the dataset into training and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_subset, val_subset = random_split(trainset, [train_size, val_size])

# load the training and validation datasets using DataLoader
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)

# Download and load the test set
testset = datasets.FashionMNIST('./data', download=True, train=False, transform=transform)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

In [36]:
class LeNet5(nn.Module):
    def __init__(self, dropout=False, batch_norm=False):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.dropout = nn.Dropout(0.5) if dropout else nn.Identity()
        self.batch_norm1 = nn.BatchNorm2d(6) if batch_norm else nn.Identity()
        self.batch_norm2 = nn.BatchNorm2d(16) if batch_norm else nn.Identity()

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.batch_norm1(x) # no-op if batch_norm is False
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = self.batch_norm2(x) # no-op if batch_norm is False
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 16*4*4) # can be replaced with torch.flatten(x, start_dim=1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout(x) # no-op if dropout is False
        x = self.fc3(x)
        return x

In [38]:
def train_model(model, loss_fn, optimizer, epochs=10, measure_without_dropout=False):
    best_val_accuracy = 0.0
    best_model_state = None
    train_accuracies, val_accuracies, test_accuracies = [], [], []

    for epoch in range(epochs):
        model.train()
        correct = 0
        total = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            # Compute the loss and its gradients
            loss = loss_fn(outputs, labels)
            loss.backward()
            # Adjust learning weights
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        train_accuracy = 100 * correct / total

        # for model using dropout regularization, go over the training set and measure the accuracy wo dropout 
        if measure_without_dropout:
            train_accuracy = evaluate_model(model, train_loader)
            model.train()
    
        train_accuracies.append(train_accuracy)

        # Validation phase
        val_accuracy = evaluate_model(model, val_loader)
        val_accuracies.append(val_accuracy)
        
        # test the model for each epoch
        test_accuracy = evaluate_model(model, test_loader)
        test_accuracies.append(test_accuracy)

        print(f"Epoch {epoch+1}/{epochs}, Train Accuracy: {train_accuracy}%, Validation Accuracy: {val_accuracy}%, Test Accuracy: {test_accuracy}%")

        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()

    # Load the best model state
    model.load_state_dict(best_model_state)
    return model, train_accuracies, val_accuracies, test_accuracies

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    # Disable gradient computation and reduce memory consumption
    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3

model = LeNet5()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#TODO: return type is different now and we need to graph the results with both training and test accuracies
train_accuracies_no_reg = train_model(model, loss_fn, optimizer, 10)
test_accuracies_no_reg = evaluate_model(model, test_loader)

In [ ]:
model_dropout = LeNet5(dropout=True)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#TODO: return type is different now and we need to graph the results with both training and test accuracies
train_accuracies_dropout = train_model(model_dropout, loss_fn, optimizer, measure_without_dropout=True)
test_accuracies_dropout = evaluate_model(model_dropout, test_loader)

In [ ]:
model_batchnorm = LeNet5(batch_norm=True)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#TODO: return type is different now and we need to graph the results with both training and test accuracies
train_accuracies_batchnorm = train_model(model_batchnorm, loss_fn, optimizer)
test_accuracies_batchnorm = evaluate_model(model_batchnorm, test_loader)

In [ ]:
model = LeNet5()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.01)

#TODO: return type is different now and we need to graph the results with both training and test accuracies
train_accuracies_weight_decay = train_model(model, loss_fn, optimizer)
test_accuracies_weight_decay = evaluate_model(model, test_loader)

In [ ]:
epochs = range(1, 20)

# Plotting convergence graphs
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(epochs, train_accuracies_no_reg, 'r', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_no_reg]*len(epochs), 'r--', label='Test Accuracy')
plt.title('No Regularization')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs, train_accuracies_dropout, 'g', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_dropout]*len(epochs), 'g--', label='Test Accuracy')
plt.title('Dropout')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(epochs, train_accuracies_batchnorm, 'b', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_batchnorm]*len(epochs), 'b--', label='Test Accuracy')
plt.title('Batch Normalization')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(epochs, train_accuracies_weight_decay, 'y', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_weight_decay]*len(epochs), 'y--', label='Test Accuracy')
plt.title('Weight Decay')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()